In [ ]:
import os
os.environ['SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL'] = 'True'

In [ ]:
!git clone https://github.com/jianhuupenn/ItClust/tree/master/tutorial/data/pancreas.git

In [ ]:
pip install keras==2.2.4 tensorflow==1.14.0

In [ ]:
pip show anndata numpy pandas tensorflow keras scipy scanpy natsort sklearn

In [1]:
import ItClust as ic
import scanpy.api as sc
import os
from numpy.random import seed
from tensorflow import set_random_seed
import pandas as pd
import numpy as np
import warnings
os.environ["CUDA_VISIBLE_DEVICES"]="1"
warnings.filterwarnings("ignore")
#import sys
#!{sys.executable} -m pip install 'scanpy==1.4.4.post1'
#Set seeds
seed(20180806)
np.random.seed(10)
set_random_seed(20180806) # on GPU may be some other default

Using TensorFlow backend.
/home/xs333/.conda/envs/ItClust/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/xs333/.conda/envs/ItClust/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/xs333/.conda/envs/ItClust/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/xs333/.conda/envs/ItClu

In [2]:
import warnings
# Suppress specific ImportWarning
warnings.filterwarnings("ignore")

## Import the data

In [3]:
#1 Read the expression matrix from *.mtx file.

# The rows of this matrix correspond to cells, columns corresond to genes. 
adata = sc.read_mtx('./10x_output/matrix.mtx')

#2 Read the *.tsv file for gene annotations. Make sure the gene names are unique.
genes = pd.read_csv('./10x_output/genes.tsv', header=None, sep='\t')
adata.var['gene_ids'] = genes[0].values
# adata.var['gene_symbols'] = genes[1].values
# adata.var_names = adata.var['gene_symbols']
# Make sure the gene names are unique
adata.var_names_make_unique(join="-")

In [4]:
adata

AnnData object with n_obs × n_vars = 30302 × 36162 
    var: 'gene_ids'

In [ ]:
import anndata
adata.obs = pd.read_csv("./labels_for_ItClust/full_annotated_supervised_macaque_obs.csv", sep=',', index_col=0)
adata

AnnData object with n_obs × n_vars = 30302 × 36162 
    obs: 'batch', 'sample', 'macaque_id', 'nGene', 'nTranscripts', 'cluster', 'region', 'class', 'cell_type', 'BATCH', 'n_genes', 'n_counts', 'new_labels', 'celltype'
    var: 'gene_ids'
    layers: 'counts'

In [48]:
adata.obs['celltype']

index
M1Fovea1_AAACCTGAGATATACG-1       DB3b
M1Fovea1_AAACCTGCACCGTTGG-1        FMB
M1Fovea1_AAACGGGAGAAGGGTA-1        IMB
M1Fovea1_AAACGGGCAAATCCGT-1       DB5*
M1Fovea1_AAACGGGCATGCGCAC-1        DB4
                                  ... 
M4PerCD73S2_TTTGTCAAGCGCCTTG-1      RB
M4PerCD73S2_TTTGTCAAGTGAATTG-1      RB
M4PerCD73S2_TTTGTCAGTGACTACT-1      RB
M4PerCD73S2_TTTGTCATCACCATAG-1      RB
M4PerCD73S2_TTTGTCATCTGTTTGT-1    DB5*
Name: celltype, Length: 30302, dtype: object

## Common Code

In [49]:
adata.raw = adata
adata.layers["counts"] = adata.X.copy()

In [50]:
print(adata)

AnnData object with n_obs × n_vars = 30302 × 36162 
    obs: 'batch', 'sample', 'macaque_id', 'nGene', 'nTranscripts', 'cluster', 'region', 'class', 'cell_type', 'BATCH', 'n_genes', 'n_counts', 'new_labels', 'celltype'
    var: 'gene_ids'
    layers: 'counts'


In [51]:
raw_adata = adata.raw.to_adata()

In [52]:
raw_adata.var_names = [name.decode('utf-8') if isinstance(name, bytes) else name for name in raw_adata.var_names]

In [53]:
raw_adata.obs_names = [name.decode('utf-8') if isinstance(name, bytes) else name for name in raw_adata.obs_names]

In [54]:
raw_adata.var_names

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '36152', '36153', '36154', '36155', '36156', '36157', '36158', '36159',
       '36160', '36161'],
      dtype='object', length=36162)

In [55]:
raw_adata.obs_names

Index(['M1Fovea1_AAACCTGAGATATACG-1', 'M1Fovea1_AAACCTGCACCGTTGG-1',
       'M1Fovea1_AAACGGGAGAAGGGTA-1', 'M1Fovea1_AAACGGGCAAATCCGT-1',
       'M1Fovea1_AAACGGGCATGCGCAC-1', 'M1Fovea1_AAACGGGGTGTTCTTT-1',
       'M1Fovea1_AAAGATGAGGGTATCG-1', 'M1Fovea1_AAAGATGCAGATGAGC-1',
       'M1Fovea1_AAAGATGTCCTTCAAT-1', 'M1Fovea1_AAAGCAAAGTGAATTG-1',
       ...
       'M4PerCD73S2_TTTGCGCCACTCGACG-1', 'M4PerCD73S2_TTTGCGCGTTCGGCAC-1',
       'M4PerCD73S2_TTTGGTTAGCCAGTAG-1', 'M4PerCD73S2_TTTGGTTCAGATCCAT-1',
       'M4PerCD73S2_TTTGGTTTCCTCTAGC-1', 'M4PerCD73S2_TTTGTCAAGCGCCTTG-1',
       'M4PerCD73S2_TTTGTCAAGTGAATTG-1', 'M4PerCD73S2_TTTGTCAGTGACTACT-1',
       'M4PerCD73S2_TTTGTCATCACCATAG-1', 'M4PerCD73S2_TTTGTCATCTGTTTGT-1'],
      dtype='object', length=30302)

In [56]:
raw_adata.raw = raw_adata

In [57]:
adata_train = raw_adata.raw.to_adata()

In [58]:
adata_test = raw_adata.raw.to_adata()

In [59]:
adata_train.raw = adata_train

In [60]:
adata_test.raw = adata_test

In [61]:
print(adata_train.raw.var_names)

Index(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       ...
       '36152', '36153', '36154', '36155', '36156', '36157', '36158', '36159',
       '36160', '36161'],
      dtype='object', length=36162)


In [62]:
print(adata_train.raw.obs_names)

Index(['M1Fovea1_AAACCTGAGATATACG-1', 'M1Fovea1_AAACCTGCACCGTTGG-1',
       'M1Fovea1_AAACGGGAGAAGGGTA-1', 'M1Fovea1_AAACGGGCAAATCCGT-1',
       'M1Fovea1_AAACGGGCATGCGCAC-1', 'M1Fovea1_AAACGGGGTGTTCTTT-1',
       'M1Fovea1_AAAGATGAGGGTATCG-1', 'M1Fovea1_AAAGATGCAGATGAGC-1',
       'M1Fovea1_AAAGATGTCCTTCAAT-1', 'M1Fovea1_AAAGCAAAGTGAATTG-1',
       ...
       'M4PerCD73S2_TTTGCGCCACTCGACG-1', 'M4PerCD73S2_TTTGCGCGTTCGGCAC-1',
       'M4PerCD73S2_TTTGGTTAGCCAGTAG-1', 'M4PerCD73S2_TTTGGTTCAGATCCAT-1',
       'M4PerCD73S2_TTTGGTTTCCTCTAGC-1', 'M4PerCD73S2_TTTGTCAAGCGCCTTG-1',
       'M4PerCD73S2_TTTGTCAAGTGAATTG-1', 'M4PerCD73S2_TTTGTCAGTGACTACT-1',
       'M4PerCD73S2_TTTGTCATCACCATAG-1', 'M4PerCD73S2_TTTGTCATCTGTTTGT-1'],
      dtype='object', length=30302)


In [63]:
clf=ic.transfer_learning_clf()
clf.fit(adata_train, adata_test)

the var_names of adata.raw: adata.raw.var_names.is_unique=: True
the var_names of adata.raw: adata.raw.var_names.is_unique=: True
Number of genes used:  2000 out of  21221
The number of training celltypes is:  13
Training the source network
The layer numbers are[128, 32]
The shape of xtrain is:30302:2000
The shape of xtest is:30302:2000
Doing DEC: pretrain
...Pretraining...
Doing SAE: pretrain_stacks
Pretraining the 1th layer...
learning rate = 0.1
learning rate = 0.01
learning rate = 0.001
The 1th layer has been pretrained.
Pretraining the 2th layer...
learning rate = 0.1
learning rate = 0.01
learning rate = 0.001
The 2th layer has been pretrained.
Doing SAE: pretrain_autoencoders
Copying layer-wise pretrained weights to deep autoencoders
Fine-tuning autoencoder end-to-end
learning rate = 0.1
learning rate = 0.010000000000000002
learning rate = 0.001
learning rate = 0.0001
learning rate = 1e-05
learning rate = 1.0000000000000002e-06
Pretraining time:  457.6066315174103
y known, initil

## Print celltype annotation

In [ ]:
celltype_pred = {}
source_label=pd.Series(clf.adata_train.obs["celltype"], dtype="category")
source_label=source_label.cat.categories.tolist()
num_ori_ct=clf.adata_test.obsm["prob_matrix"+str(clf.save_atr)].shape[1]
target_label=[str(i) for i in range(num_ori_ct)]
for i in range(num_ori_ct):
    end_cell=clf.adata_test.obs.index[clf.adata_test.obs["decisy_trans_True"]==target_label[i]]
    start_cell=clf.adata_test.obs.index[clf.adata_test.obs["trajectory_0"]==target_label[i]]
    overlap=len(set(end_cell).intersection(set(start_cell)))
    celltype_pred[target_label[i]]=[source_label[i], round(overlap/(len(end_cell)+0.0001),3)]

In [ ]:
pred, prob, celltype_pred = clf.predict()
pred.head()

## Save the embeddings

In [64]:
embeddings_dict = {}

In [65]:
embeddings_dict['macaque'] = clf.adata_test.obsm["X_Embeded_z"+str(clf.save_atr)]

In [66]:
embeddings_dict['macaque']

array([[-0.9738285 , -0.99998844,  0.999527  , ...,  0.9997807 ,
         0.9999094 , -0.9993435 ],
       [-0.99840933,  0.99847394,  0.9995961 , ..., -0.99992794,
        -0.9991605 , -0.99834406],
       [-0.99732256, -0.99830425,  0.9990034 , ...,  0.9994985 ,
        -0.9984777 , -0.99914724],
       ...,
       [ 0.99985707,  0.99998045, -0.99999785, ..., -0.99912274,
         0.9993621 ,  0.9994434 ],
       [ 0.99990743,  0.9999263 , -0.99999845, ..., -0.9980665 ,
         0.99959135,  0.99926156],
       [-0.7022455 ,  0.99963284,  0.99987644, ..., -0.9994949 ,
        -0.9986844 ,  0.9992414 ]], dtype=float32)

In [ ]:
combined_embeddings = pd.concat(
    {key:pd.DataFrame(value, index=adata_test.obs_names) for key, value in embeddings_dict.items()}, axis=1
)

combined_embeddings.to_csv("./results/full_annotated_supervised_macaque_itclust_embeddings.csv")